# Comparative between different models for TED Talks dataset

In [1]:
%reload_ext autoreload

%autoreload 2

In [2]:
from Libraries_functions import *
from tqdm import tqdm

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

mirrored_strategy = tf.distribute.MirroredStrategy()

# MAIN

## This notebook loads data from folders directly to memory

## Load data

In [3]:

##Load original load function
#df_data,df_route = load_df('D:\Caracuel\WaveNet-seaBackGroundNoise\Datasets\Data\dataframe_large.csv')
df_data,df_route = load_df('D:\Caracuel\WaveNet-seaBackGroundNoise\Code\generatingData\dataframe.csv')
#df_data,df_route = load_df('./dataframe.csv')

##Load data with read wavs
#df_data=read_wavs("D:\\Caracuel\\WaveNet-seaBackGroundNoise\\Datasets\\Noise\\Chunked_large\\","D:\\Caracuel\\WaveNet-seaBackGroundNoise\\Datasets\\Audio_mezclado_large\\")
#df_data=read_wavs("D:\\Caracuel\\WaveNet-seaBackGroundNoise\\Datasets\\Pruebas_pequeñas_noise\\","D:\\Caracuel\\WaveNet-seaBackGroundNoise\\Datasets\\Pruebas_pequeñas_mixed\\")


##Load paths of data for generator
#df_data=read_path("D:\\Caracuel\\WaveNet-seaBackGroundNoise\\Datasets\\Noise\\Chunked_large\\","D:\\Caracuel\\WaveNet-seaBackGroundNoise\\Datasets\\Audio_mezclado_large\\")
#create train, validate and test datasets
#train, validate, test = np.split(df_data.sample(frac=1), [int(.6*len(df_data)), int(.8*len(df_data))])



Opening CSV...

Reading data...

Preprocessing data


100%|██████████| 2000/2000 [00:49<00:00, 40.59it/s]


Finished successfully!


miramos cual es la longitud más pequeña por si se ha descuadrado algo

In [4]:
shortest_length = min(map(len, df_data.data))

Utilizamos solo los datos de lluvia para entrenar el modelo

In [5]:
##
# df_lluvia=pd.DataFrame()
# lluvia_mezclada=[]
# lluvia=[]
# for i in tqdm(range(len(df_data))):
#     if(i>=df_data.shape[0]/4-1 and i<df_data.shape[0]/4+499-1):
#         lluvia.append(df_data.data[i])
#         lluvia_mezclada.append(df_data.noise_original[i])

# df_lluvia['data']=lluvia
# df_lluvia['noise_original']=lluvia

### Wave U-Net

Lets prepare the train and test data for the model.

For the Wave U-Net model, all files are going to be chuncked to 32k frames due to the model input and output shapes.

Calculamos cuanto tiene que medir la red en funcion del tamaño de la entrada. Ajustaremos dicho dato lo más posible a la entrada original.

In [6]:
inp=shortest_length-shortest_length%16

print(inp)

32000


In [7]:
X, y = chunk4unet(df_data, 'data', 'noise_original',inp)

del df_data
gc.collect()

Chunking data to common format


107

In [8]:
np.shape(X)

(2000, 32000)

In [9]:
X = np.reshape(X, (np.shape(X)[0], np.shape(X)[1], 1))
y = np.reshape(y, (np.shape(y)[0], np.shape(y)[1], 1))

In [10]:
print(len(y[0]))
print(len(X[0]))
print(np.shape(X))

32000
32000
(2000, 32000, 1)


### Creamos los datasets de entrenamiento, validacion y test

In [11]:
X_train, X_tes, y_train, y_tes = train_test_split(X,y, test_size=0.1)

del X
del y
gc.collect()

86

### Instanciamos el generador creado a mano y lo utilizamos

In [12]:
# traingen=CustomDataGen(train,1920000,5)

In [13]:
# valgen=CustomDataGen(validate,1920000,5)

### Creamos el generador de tensroflow de datos para que quepan todos con un mismo batch

In [14]:
# trainx_dataset = tf.data.Dataset.from_tensor_slices((X_train))
# trainy_dataset = tf.data.Dataset.from_tensor_slices((y_train))
# # train_dataset=tf.data.Dataset.from_tensor_slices((X_train,y_train))
# # val_dataset=tf.data.Dataset.from_tensor_slices((X_val,y_val))
# train_dataset = tf.data.Dataset.zip((trainx_dataset, trainy_dataset))
# valx_dataset = tf.data.Dataset.from_tensor_slices(X_val)
# valy_dataset = tf.data.Dataset.from_tensor_slices(y_val)
# val_dataset = tf.data.Dataset.zip((valx_dataset, valy_dataset))

In [15]:
#  probar la red con lo hecho ahora con los tensor slices
# np.shape(list(train_dataset.as_numpy_iterator()))

In [16]:
sistema = platform.system()
if (sistema =="Linux"):
        # Open a strategy scope.
    with mirrored_strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
        # Call to the original u-Net
        model = unet(inp)
    adam=tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=adam, loss='mse', metrics=['accuracy'])
else:
    model = unet(inp)
    adam = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=adam, loss='mse', metrics=['accuracy'])
    model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32000, 1)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 32000, 64)    256         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 32000, 64)    12352       conv1d[0][0]                     
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 16000, 64)    0           conv1d_1[0][0]                   
_______________________________________________________________________________________

In [17]:
path="./"
checkpoint = [callbacks.EarlyStopping(patience=5, restore_best_weights=True),callbacks.ModelCheckpoint(
    path,
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    mode="max"
)]

history=model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=10,
    validation_split=0.2,
    verbose=1,
    callbacks=[checkpoint]
)

Epoch 1/50
 17/144 [==>...........................] - ETA: 34:44 - loss: 0.9991 - accuracy: 0.0568

Predict model

In [ ]:
plot_accuracy_loss(history,1)

pred_outp=model.predict(X_test)

print(np.shape(pred_outp))
audio_output = np.reshape(pred_outp, 1,(np.shape(pred_outp)))

export_wav(pred_outp)